In [1]:
# 1 导入需要的模块
import torchvision
import torch
import torch.utils.data as data
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torch.nn as nn
from torch.optim import sgd, adam
from PIL import Image
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import os
import shutil

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
transform_train = transforms.Compose(
    [
        transforms.Resize((256, 256)),
        transforms.RandomCrop((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])

    ]
)

transform_test = transforms.Compose(
    [
        transforms.Resize((256, 256)),
        transforms.RandomCrop((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])

    ]
)


In [3]:
# 3 训练集和测试集的上一级路径 
image_path = "/content/drive/My Drive" 
# image_path = "D:\\迅雷下载\\dogs-vs-cats-redux-kernels-edition"  # windows上的路径 

In [4]:
# 4 训练集和测试集内容以及标签获取类 
class CatDogProcessor(data.Dataset):
    def __init__(self, mode, directory):
        self.mode = mode
        self.list_img = []
        self.list_label = []
        self.data_size = 0

        if self.mode == "train":
            self.transform = transform_train
            directory = directory + "/sdh_train/"
            for file in os.listdir(directory):
                self.list_img.append(directory + file)
                self.data_size += 1
                name = file.split(sep=".")
                if name[0] == "cat":
                    self.list_label.append(0)
                elif name[0] == "dog":
                    self.list_label.append(1)

        elif self.mode == "test":
            self.transform = transform_test
            directory = directory + "/sdh_test/"
            for file in os.listdir(directory):
                self.list_img.append(directory + file)
                self.data_size += 1
                name = file.split(sep=".")
                if name[0] == "cat":
                    self.list_label.append(0)
                elif name[0] == "dog":
                    self.list_label.append(1)

    def __getitem__(self, item):
        if self.mode == "train":
            img = Image.open(self.list_img[item])
            label = self.list_label[item]
            return self.transform(img), torch.tensor([label])
        elif self.mode == "test":
            img = Image.open(self.list_img[item])
            label = self.list_label[item]
            return self.transform(img), torch.tensor([label])
        else:
            return None

    def __len__(self):
        return self.data_size

In [6]:
 # 5 定义一个神经网络模型
class SelfMadeCNN(nn.Module):
    # 5个卷积层＋2个全连接层
    def __init__(self, n_hidden_1, n_hidden_2,out_dim):
        super(SelfMadeCNN, self).__init__()
        self.layer1 = nn.Sequential(
            # (224-3+2*1)/1 +1 = 224
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),  # 批标准化放在卷积层后面，激活层前面
            nn.ReLU(inplace=True),
            # 224/2 = 112
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.layer2 = nn.Sequential(
            # (112-3+2*0)/1 +1 = 110
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=0),
            nn.BatchNorm2d(64),  # 批标准化放在卷积层后面，激活层前面
            nn.ReLU(inplace=True),
            # (110-2)/2 +1 = 55
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.layer3 = nn.Sequential(
            #  55-2+2*0+1=54
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=2, stride=1, padding=0),
            nn.BatchNorm2d(64),  # 批标准化放在卷积层后面，激活层前面
            nn.ReLU(inplace=True),
            # (54-2)/2 +1 = 27
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.layer4 = nn.Sequential(
            #  27-2+2*0+1=26
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=2, stride=1, padding=0),
            nn.BatchNorm2d(64),  # 批标准化放在卷积层后面，激活层前面
            nn.ReLU(inplace=True),
            # (26-2)/2 +1 = 13
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.layer5 = nn.Sequential(
            #  13-2+2*0+1=12
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=2, stride=1, padding=0),
            nn.BatchNorm2d(64),  # 批标准化放在卷积层后面，激活层前面
            nn.ReLU(inplace=True),
            # (12-2)/2 +1 = 6
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.layer6 = nn.Sequential(
            nn.Dropout(p=0.5),
            nn.Linear(6 * 6 * 64, n_hidden_1),
            nn.ReLU(inplace=True),

            nn.Dropout(p=0.5),
            nn.Linear(n_hidden_1, n_hidden_2),
            nn.ReLU(inplace=True),
            
            nn.Linear(n_hidden_2, out_dim),

        )

    def forward(self, x):
        # 卷积层
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)

        # 全连接层
        # 进入全连接之前要将矩阵转为向量
        x = x.view(x.size(0), -1)
        x = self.layer6(x)

        return x

In [8]:
def plot_image_train(acces, train_losses):
    plt.plot(np.arange(len(acces)), acces)
    plt.title('train acc')
    plt.show()

    plt.plot(np.arange(len(train_losses)), train_losses)
    plt.title('train loss')
    plt.show()

def plot_image_test(acces, test_losses):
    plt.plot(np.arange(len(acces)), acces)
    plt.title('test acc')
    plt.show()

    plt.plot(np.arange(len(test_losses)), test_losses)
    plt.title('test loss')
    plt.show()

In [9]:
def train():
    train_image_data_set = CatDogProcessor("train", image_path)
    train_data = DataLoader(train_image_data_set, batch_size=16, shuffle=True)
    print("train_data loaded", len(train_data))

    test_image_data_set = CatDogProcessor("test", image_path)
    test_data = DataLoader(test_image_data_set, batch_size=16, shuffle=True)
    print("train_data loaded", len(test_data))

    model = SelfMadeCNN(100, 100, 2)
    model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = sgd.SGD(model.parameters(), lr=0.001,momentum=0.9)

    train_losses = []
    train_acces = []
    test_losses = []
    test_acces = []

    for i in range(300):
        train_loss = 0
        train_acc = 0
        model.train()
        for im, label in train_data:
            # 前向传播
            im = im.cuda()
            label = label.cuda()
            label = label.squeeze()
            out = model(im)
            loss = criterion(out, label)

            # 反向传播
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # 记录误差
            train_loss += loss.item()
            # 计算分类的准确率
            _, pred = torch.max(out, 1)
            num_correct = (pred == label).sum().item()
            acc = num_correct / im.shape[0]
            train_acc += acc
        train_losses.append(train_loss / len(train_data))
        train_acces.append(train_acc / len(train_data))

        test_loss = 0
        test_acc = 0
        model.eval()
        for im_test, label_test in test_data:
            # 前向传播
            im_test = im_test.cuda()
            label_test = label_test.cuda()
            label_test = label_test.squeeze()

            out_test = model(im_test)
            loss_test = criterion(out_test, label_test)

            # 记录误差
            test_loss += loss_test.item()
            # 计算分类的准确率
            _, pred_test = torch.max(out_test, 1)
            num_correct_test = (pred_test == label_test).sum().item()
            acc_test = num_correct_test / im_test.shape[0]
            test_acc += acc_test

        test_losses.append(test_loss / len(test_data))
        test_acces.append(test_acc / len(test_data))

        print("epoch: %d, train loss: %.6f, train acc: %.6f, test loss: %.6f, test acc: %.6f" % (
            i, train_loss / len(train_data), train_acc / len(train_data), test_loss / len(test_data),
            test_acc / len(test_data)))

    print("train and test-----------------Done!")

    plot_image_train(train_acces, train_losses)
    plot_image_test(test_acces, test_losses)

    torch.save(model.state_dict(), "model.pth")

![jupyter](./demo34.png)

![jupyter](./demo34_2.png)